# pFBA simulation for the MT-GSM of Poplar tricocarpa
## Original Code: Juliana Simas Barbosa
## Edits/streamlining: Wheaton Schroeder
### Latest Version: 08/28/2023

#### Make library imports

In [1]:
# Import necessary libraries:
import cobra
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import warnings

#### Define necessary functions

#### set information for the runs

In [4]:
# initial inputs:
## mass (gDW) of each tissue in year one 
### Leaf, stem, root
#note that these values will be the same for colntorl and drought, as prolonged drought introduced in the growing season modeled

#initial masses, from a paper
#aaron hogan et al 2021 or 2020
Init_S_mass = 185
Init_R_mass = 112
Init_L_mass = 121
total_mass = Init_S_mass + Init_R_mass + Init_L_mass

#fitted starts, from fitting of growth curves
#Init_S_mass = 100.54
#Init_R_mass = 67.403
#Init_L_mass = 86.659
#total_mass = Init_S_mass + Init_R_mass + Init_L_mass

#growth rates for the control model, calculated by fitting an experimental function
cont_leaf_g = 0.001946356
cont_root_g = 0.002960933
cont_stem_g = 0.003555685

#mass fractions at first time point
L_fraction_0 = Init_L_mass/total_mass
R_fraction_0 = Init_R_mass/total_mass
S_fraction_0 = Init_S_mass/total_mass

#mass values at end of simulated growing season
stem_fitted_1 = Init_S_mass * np.exp(cont_stem_g * 171) 
root_fitted_1 = Init_R_mass * np.exp(cont_root_g * 171) 
leaf_fitted_1 = Init_L_mass * np.exp(cont_leaf_g * 171)
total_fitted_1 = stem_fitted_1 + root_fitted_1 + leaf_fitted_1

#we need to define leaf to shoot (sigma) and root to shoot (gamma) values based on the next time point for 
sigma = leaf_fitted_1 / stem_fitted_1
gamma = 2 * root_fitted_1 / stem_fitted_1

#photon uptake limits
stem_photon_uptake_ub = 443
leaf_photon_uptake_ub = 443

#non-growth associated maintenance
NGAM = 23.07238

#control model file
control_file = "./iPotri3016C.xml"

print("success")

success


#calculate the required stem growth rate
v_stem = v_root = (np.log())

#### read the model

In [5]:
# Load the model
control_model = cobra.io.read_sbml_model(control_file)

print("success")

No objective coefficients in model. Unclear what should be optimized


success


#### update model intertissue transport reactions

In [6]:
# Load inter-tissue transport reaction identifiers
it_ids = open('intertissue_transport_ids.txt')
IT_rxnIDs = it_ids.read().splitlines()
it_ids.close()

# Load inter-tissue transport reaction equations
it_id_eqs = open('intertissue_transport_eqs.txt')
IT_rxnEQs = it_id_eqs.read().splitlines()
it_id_eqs.close()

rxn_list = [False]*len(IT_rxnIDs)

# Extract the cobra Reaction object corresponding to the inter-tissue transport reactions
for idx,rxnID in enumerate(IT_rxnIDs):

  #rxn_list becomes an list of reaction objects
  rxn_list[idx] = control_model.reactions.get_by_id(rxnID) 

# Edit the reaction stoichiometric coefficients to account for tissue fractions at different time-points
for idx, rxns in enumerate(rxn_list):

  #for debugging
  #print(idx)
  #turns out idx is index of reaction in the temp

  temp_halves = IT_rxnEQs[idx].split(' <=> ')
  
  if 'R2CP1' in str(IT_rxnIDs[idx]):   
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(R_fraction_0)+' '+temp_halves[1])
  
  if 'S2CP1' in str(IT_rxnIDs[idx]):
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(S_fraction_0)+' '+temp_halves[1])
  
  if 'S2CP2' in str(IT_rxnIDs[idx]):
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(S_fraction_0)+' '+temp_halves[1])
  
  if 'L2CP2' in str(IT_rxnIDs[idx]):
    rxns.build_reaction_from_string('1 '+temp_halves[0]+' <=> '+str(L_fraction_0)+' '+temp_halves[1])
        
  #for debugging
  #print(rxns)
  
  # Add the reactions back to the model
  #note: this will throw a lot of "ignoring reaction '...' since it already exists" warnings. It does appear that this does update the stoichiometry (I checked the stoichiometry of RXN_L2CP2_SUCROSE DARK before and after) this code block and the stoichiometry is updated, the warning I think
  
  #update the reaction stoichiometry
  control_model.add_reactions([rxns])

  #give bounds allowing flow of metabolites in either direction
  control_model.reactions.get_by_id(rxns.id).lower_bound = -10000; control_model.reactions.get_by_id(rxns.id).upper_bound = 10000

print("success")

FileNotFoundError: [Errno 2] No such file or directory: 'intertissue_transport_ids.txt'

#### Add pysical/sensible constraints to the model

In [ ]:
#diurnal or sensible restrictions on flow of metabolites across system boundary
## NIGHT restrictions
control_model.reactions.RXN_E2L_CO2_DARK.lower_bound = -10000;control_model.reactions.RXN_E2L_CO2_DARK.upper_bound = 0              #CO2 export from leaf (heterotrophic growth)
control_model.reactions.RXN_E2L_LIGHT__L___DARK.lower_bound = 0;control_model.reactions.RXN_E2L_LIGHT__L___DARK.upper_bound = 0     #no light uptake in leaf (heterotrophic growth)
control_model.reactions.RXN_E2S_LIGHT__S___DARK.lower_bound = 0;control_model.reactions.RXN_E2S_LIGHT__S___DARK.upper_bound = 0     #no light uptake in stem (heterotrophic growth)
control_model.reactions.RXN_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p__L___DARK.upper_bound = 0                                       #turns off rubisco CO2 fixation to be safe
control_model.reactions.RXN_E2S_CO2_DARK.lower_bound = 0;control_model.reactions.RXN_E2S_CO2_DARK.upper_bound = 10000               #stem exports CO2 during night

## DAY restrictions
#restrict CO2 uptake to 23.4 mmol/gDWday as seen experimentally, determine NGAM from that
control_model.reactions.RXN_E2L_CO2_LIGHT.lower_bound = 0;control_model.reactions.RXN_E2L_CO2_LIGHT.upper_bound = 23.4               #leaf uptakes CO2 during day (phototrophic metabolism)
control_model.reactions.RXN_E2L_LIGHT__L___LIGHT.lower_bound = 0;control_model.reactions.RXN_E2L_LIGHT__L___LIGHT.upper_bound = 10000   #leaf uptakes light during day
control_model.reactions.RXN_E2S_LIGHT__S___LIGHT.lower_bound = 0;control_model.reactions.RXN_E2S_LIGHT__S___LIGHT.upper_bound = stem_photon_uptake_ub   #stem can uptake some limited light during day
control_model.reactions.RXN_E2S_CO2_LIGHT.lower_bound = 0;control_model.reactions.RXN_E2S_CO2_LIGHT.upper_bound = 10000             #stem exports CO2 during day

print("success")

#### build model constraints, control model needs 2-3, 11-12, and 14-20

In [ ]:
# mass balance constraint should be automatically enforced by COBRA (equation 2)
# reaction flux bound constraints should be automatically enforced by COBRA (equation 3)

## enforce experimentally observed growth rates (equation 11)
#growth only at night
control_model.reactions.RXN_BiomassRxn__L___DARK.lower_bound = cont_leaf_g; control_model.reactions.RXN_BiomassRxn__L___DARK.upper_bound = cont_leaf_g
control_model.reactions.RXN_BiomassRxn__S___DARK.lower_bound = cont_stem_g; control_model.reactions.RXN_BiomassRxn__S___DARK.upper_bound = cont_stem_g
control_model.reactions.RXN_BiomassRxn__R___DARK.lower_bound = cont_root_g; control_model.reactions.RXN_BiomassRxn__R___DARK.upper_bound = cont_root_g

#so turn off growth in the light
control_model.reactions.RXN_BiomassRxn__L___LIGHT.lower_bound = 0; control_model.reactions.RXN_BiomassRxn__L___LIGHT.upper_bound = 0
control_model.reactions.RXN_BiomassRxn__S___LIGHT.lower_bound = 0; control_model.reactions.RXN_BiomassRxn__S___LIGHT.upper_bound = 0
control_model.reactions.RXN_BiomassRxn__R___LIGHT.lower_bound = 0; control_model.reactions.RXN_BiomassRxn__R___LIGHT.upper_bound = 0

#require that all NGAM values are equal
ngam_equal_1_dark = control_model.problem.Constraint(control_model.reactions.RXN_ATPM_c__L___DARK.flux_expression - control_model.reactions.RXN_ATPM_c__S___DARK.flux_expression, ub = 0, lb = 0)
ngam_equal_2_dark = control_model.problem.Constraint(control_model.reactions.RXN_ATPM_c__L___DARK.flux_expression - control_model.reactions.RXN_ATPM_c__R___DARK.flux_expression, ub = 0, lb = 0)
ngam_equal_1_light = control_model.problem.Constraint(control_model.reactions.RXN_ATPM_c__L___LIGHT.flux_expression - control_model.reactions.RXN_ATPM_c__S___LIGHT.flux_expression, ub = 0, lb = 0)
ngam_equal_2_light = control_model.problem.Constraint(control_model.reactions.RXN_ATPM_c__L___LIGHT.flux_expression - control_model.reactions.RXN_ATPM_c__R___LIGHT.flux_expression, ub = 0, lb = 0)
ngam_equal_dark_light = control_model.problem.Constraint(control_model.reactions.RXN_ATPM_c__L___LIGHT.flux_expression - control_model.reactions.RXN_ATPM_c__L___DARK.flux_expression, ub = 0, lb = 0)

control_model.add_cons_vars(ngam_equal_1_dark)
control_model.add_cons_vars(ngam_equal_2_dark)
control_model.add_cons_vars(ngam_equal_1_light)
control_model.add_cons_vars(ngam_equal_2_light)
control_model.add_cons_vars(ngam_equal_dark_light)

## Imposes that the transport of CO2 from the stem xylem to leaves is at most 2.7% of absorbed atmospheric CO2 (equation 14)
#RXN_L2CP2_CO2_LIGHT is positive if loading CO2 into the CP2 
#RXN_E2L_CO2_LIGHT is positive if CO2 is moving into the leaf extracellular compartment from the shared extracellular compartment
co2tr = control_model.problem.Constraint(control_model.reactions.RXN_L2CP2_CO2_LIGHT.flux_expression + 0.027 * control_model.reactions.RXN_E2L_CO2_LIGHT.flux_expression, ub = 10000, lb=0)
#co2tr2 = control_model.problem.Constraint(control_model.reactions.RXN_L2CP2_CO2_DARK.flux_expression + 0.027 * control_model.reactions.RXN_E2L_CO2_DARK.flux_expression, ub = 10000, lb=0)
control_model.add_cons_vars(co2tr)
#control_model.add_cons_vars(co2tr2)

## Imposes that the amount of respired CO2 loaded onto the xylem by the roots rivals (in this case, equals) the export of CO2 to the soil (environment) (equation 15)
co2tr3 = control_model.problem.Constraint(control_model.reactions.RXN_E2R_CO2_LIGHT.flux_expression - control_model.reactions.RXN_R2CP1_CO2_LIGHT.flux_expression, ub=0, lb=0)
co2tr4 = control_model.problem.Constraint(control_model.reactions.RXN_E2R_CO2_DARK.flux_expression - control_model.reactions.RXN_R2CP1_CO2_DARK.flux_expression, ub=0, lb=0)
control_model.add_cons_vars(co2tr3)  
control_model.add_cons_vars(co2tr4) 

# limit photon uptake in the light based on calculations described in the paper (equation 16)
control_model.reactions.RXN_E2L_LIGHT__L___LIGHT.lower_bound = 0;control_model.reactions.RXN_E2L_LIGHT__L___LIGHT.upper_bound = leaf_photon_uptake_ub

## Allowing storage of respired CO2 in the stem, working toward building equation 17
control_model.add_boundary(control_model.metabolites.MET_carbon_dioxide_c__S___DARK,type = "demand")
control_model.add_boundary(control_model.metabolites.MET_carbon_dioxide_c__S___LIGHT,type = "sink")

#This prevents CO2 storage during the day
control_model.reactions.SK_MET_carbon_dioxide_c__S___LIGHT.upper_bound = 0; 

# Imposes that 10% of respired CO2 is stored into the stems during the night (equation 17)
# requiring 10% storage
co2storage = control_model.problem.Constraint(0.1*control_model.reactions.RXN_E2S_CO2_DARK.flux_expression - control_model.reactions.DM_MET_carbon_dioxide_c__S___DARK.flux_expression, ub = 0, lb = 0)
# requiring the storage at night to equal its use in day
co2storage2 = control_model.problem.Constraint(control_model.reactions.SK_MET_carbon_dioxide_c__S___LIGHT.flux_expression + control_model.reactions.DM_MET_carbon_dioxide_c__S___DARK.flux_expression, ub = 0, lb = 0)
control_model.add_cons_vars(co2storage)
control_model.add_cons_vars(co2storage2)

# Adding connecting starch reactions for creating equations 18 and 19
## Define sink and demand reactions
## sing reactions are for removal of starch from storage, demand are for adding starch to storage
control_model.add_boundary(control_model.metabolites.MET_starch_p__L___LIGHT, type="demand")
control_model.add_boundary(control_model.metabolites.MET_starch_p__S___LIGHT, type="demand")
control_model.add_boundary(control_model.metabolites.MET_starch_p__L___DARK, type="sink")
control_model.add_boundary(control_model.metabolites.MET_starch_p__S___DARK, type="sink")
    
# Only allows consumption of starch at night
control_model.reactions.SK_MET_starch_p__L___DARK.upper_bound = 0
control_model.reactions.SK_MET_starch_p__S___DARK.upper_bound = 0

# by default, demand reactions have LB = 0, so don't need to specify this to ensure only storage during day. 

## Requiring that starch producded during the day is used at night, and visa versa (equations 18 and 19)
# leaf
starchprodL = control_model.problem.Constraint(control_model.reactions.DM_MET_starch_p__L___LIGHT.flux_expression + control_model.reactions.SK_MET_starch_p__L___DARK.flux_expression, lb = 0, ub = 0)
control_model.add_cons_vars(starchprodL)

#stem
starchprodS = control_model.problem.Constraint(control_model.reactions.DM_MET_starch_p__S___LIGHT.flux_expression + control_model.reactions.SK_MET_starch_p__S___DARK.flux_expression, lb = 0, ub = 0)
control_model.add_cons_vars(starchprodS)

## Imposes a photorespiration rate of 25% during the day (equation 20)
photoresp = control_model.problem.Constraint(control_model.reactions.RXN_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p__L___LIGHT.flux_expression + 4*control_model.reactions.RXN_RXN_961_p__L___LIGHT.flux_expression, ub=0,lb=0)
control_model.add_cons_vars(photoresp)

print("success")

#### Define the model objective and direction

In [ ]:
control_model.objective = control_model.reactions.RXN_ATPM_c__S___DARK
control_model.objective_direction = 'max'

print("success")

#### Solve the pFBA problem, report results

In [ ]:
#solve model
c_solution = cobra.flux_analysis.pfba(control_model)

In [ ]:
#report solution
print("total sum of fluxes:\t\t"+str(c_solution.objective_value)+" mmol/gDWday")
print("starch production in the leaf:\t"+str(c_solution.fluxes.DM_MET_starch_p__L___LIGHT) +" mmol/gDWday")
print("starch production in the stem:\t"+str(c_solution.fluxes.DM_MET_starch_p__S___LIGHT) +" mmol/gDWday")

print("Day leaf CO2 uptake:\t\t"+ str(c_solution.fluxes.RXN_E2L_CO2_LIGHT)+" mmol/gDWday")
print("Night leaf CO2 efflux:\t\t"+ str(-c_solution.fluxes.RXN_E2L_CO2_DARK)+" mmol/gDWday")
print("Day stem CO2 efflux:\t\t"+ str(c_solution.fluxes.RXN_E2S_CO2_LIGHT)+" mmol/gDWday")
print("Night stem CO2 efflux:\t\t"+ str(c_solution.fluxes.RXN_E2S_CO2_DARK)+" mmol/gDWday")
print("Day root CO2 efflux:\t\t"+ str(c_solution.fluxes.RXN_E2R_CO2_LIGHT)+" mmol/gDWday")
print("Night root CO2 efflux:\t\t"+ str(c_solution.fluxes.RXN_E2R_CO2_DARK)+" mmol/gDWday")

total_efflux = -c_solution.fluxes.RXN_E2L_CO2_DARK + c_solution.fluxes.RXN_E2S_CO2_LIGHT + c_solution.fluxes.RXN_E2S_CO2_DARK + c_solution.fluxes.RXN_E2R_CO2_LIGHT + c_solution.fluxes.RXN_E2R_CO2_DARK
print("Total CO2 efflux:\t\t"+ str(total_efflux)+" mmol/gDWday")

net_CO2 = c_solution.fluxes.RXN_E2L_CO2_LIGHT - total_efflux

print("Net CO2 uptake:\t\t\t"+ str(net_CO2)+" mmol/gDWday")
print("\nMax ATPM:\t\t\t"+ str(c_solution.fluxes.RXN_ATPM_c__S___DARK)+" mmol/gDWday")

df = pd.DataFrame([[c_solution.fluxes[i]] for i in range(0, len(c_solution.fluxes))])
df.index = [rxn.id for rxn in control_model.reactions]
df.to_csv('pFBA_results_control.csv', sep=',', index=True)

gam_cost = 693.12 * (cont_leaf_g + cont_stem_g + cont_root_g)

print("\nGAM cost:\t\t\t"+ str(gam_cost)+" mmol/gDWday")

#### Save NGAM

In [ ]:
ngam_out = open("iPorti3016C_NGAM_est.txt","w")
ngam_out.write(str(c_solution.fluxes.RXN_ATPM_c__S___DARK))
ngam_out.close()